# Анализ временных и географических признаков

В данном ноутбуке рассматриваются методы создания и анализа временных и географических признаков, а также обучение модели на основе этих данных.

In [ ]:

# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


## Загрузка и просмотр данных

Сначала создадим тестовый датафрейм с признаками `timestamp`, `latitude`, `longitude`, и `target`.

In [ ]:

# Предположим, что у нас есть тестовый датафрейм
data = {
    'timestamp': pd.date_range('2023-01-01', periods=100, freq='H'),
    'latitude': np.random.uniform(50, 60, 100),
    'longitude': np.random.uniform(30, 40, 100),
    'target': np.random.rand(100)
}
df = pd.DataFrame(data)
df.head()


## Преобразование временных данных

Добавим временные признаки, такие как год, месяц, день, час и день недели. Также создадим циклические признаки для дня недели и часа.

In [ ]:

# Преобразование временных признаков
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek

# Циклическое представление для дня недели и часа
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Создание лаговых признаков (например, значение таргета за 1 час назад)
df['target_lag_1'] = df['target'].shift(1)
df['target_lag_2'] = df['target'].shift(2)
df['target_rolling_mean_3'] = df['target'].rolling(window=3).mean()

# Удаление строк с NaN (так как некоторые лаги не определены для первых строк)
df = df.dropna().reset_index(drop=True)
df.head()


## Преобразование географических данных

Для работы с географическими признаками преобразуем широту и долготу в трехмерное пространство. Также добавим кластеризацию с использованием KMeans для группировки регионов.

In [ ]:

# Преобразование географических признаков
df['x'] = np.cos(np.radians(df['latitude'])) * np.cos(np.radians(df['longitude']))
df['y'] = np.cos(np.radians(df['latitude'])) * np.sin(np.radians(df['longitude']))
df['z'] = np.sin(np.radians(df['latitude']))

# Пространственная кластеризация (например, KMeans)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(df[['latitude', 'longitude']])
df['location_cluster'] = kmeans.labels_

df.head()


## Визуализация временных и географических данных

Построим графики для анализа временной и пространственной информации.

In [ ]:

import seaborn as sns

# Временные циклы
plt.figure(figsize=(14,6))
sns.lineplot(data=df, x='timestamp', y='target')
plt.title('Временная зависимость целевого признака')
plt.show()

# Географические кластеры
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x='longitude', y='latitude', hue='location_cluster', palette='viridis')
plt.title('Географические кластеры')
plt.show()


## Подготовка данных для модели

Разделим данные на обучающую и тестовую выборки и обучим модель градиентного бустинга.

In [ ]:

# Определение признаков и целевого значения
X = df.drop(columns=['timestamp', 'target'])
y = df['target']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Пример: обучение модели градиентного бустинга
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE модели: {rmse}')


## Анализ важности признаков

Посмотрим на важность признаков, чтобы понять, какие из временных и географических признаков оказались наиболее значимыми.

In [ ]:

# Важность признаков
feature_importance = model.feature_importances_
features = X.columns
sorted_idx = np.argsort(feature_importance)

plt.figure(figsize=(10, 8))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), features[sorted_idx])
plt.title('Важность признаков')
plt.show()
